In [1]:
from __future__ import annotations
from data_handler import *
from analysis import *
from dsp_premitives import *
from processors import *
from render_engine import *
from streaming_normalization_simulator import *
from comparison_reporting import *
from presets_recommendations import *
from logging_versioning import *
from utils import *
from mastering_orchestrator import *


# ---------- VARIANT DEFINITIONS ----------
# Define all possible variants (recommended ones will get a "_recommended" suffix later)
ALL_VARIANTS = [
    ("Transparent_recommended", DialState(bass=0, punch=0, clarity=0, air=0, width=0)),
    ("Bassier_recommended",     DialState(bass=2, punch=0, clarity=0, air=0, width=0)),
    ("Punchier_recommended",    DialState(bass=0, punch=2, clarity=0, air=0, width=0)),
    ("Clearer_recommended",     DialState(bass=0, punch=0, clarity=2, air=0, width=0)),
    ("Airy_recommended",        DialState(bass=0, punch=0, clarity=0, air=2, width=0)),
    ("Wider_recommended",       DialState(bass=0, punch=0, clarity=0, air=0, width=2)),

    # Non-recommended exploratory variants
    ("Bass_max",     DialState(bass=4, punch=0, clarity=0, air=0, width=0)),
    ("Punch_max",    DialState(bass=0, punch=4, clarity=0, air=0, width=0)),
    ("Clarity_max",  DialState(bass=0, punch=0, clarity=4, air=0, width=0)),
    ("Air_max",      DialState(bass=0, punch=0, clarity=0, air=4, width=0)),
    ("Width_max",    DialState(bass=0, punch=0, clarity=0, air=0, width=4)),

    # Mixed tweaks
    ("Bass_and_Punch", DialState(bass=2, punch=2, clarity=0, air=0, width=0)),
    ("Air_and_Clarity", DialState(bass=0, punch=0, clarity=2, air=2, width=0)),
    ("Full_mix",       DialState(bass=2, punch=2, clarity=2, air=2, width=2)),
]


Post-Mix I/O layer loaded: AudioBuffer, load_wav, save_wav, resample_poly, slice_preview, with_suffix, auto_out_path, sha256_file, print_audio_summary.
Convenience helpers loaded: make_workspace, batch_load_wavs, env_fingerprint, Manifest, write/read_manifest, register_input, register_artifact, import_mix.
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Patched: render_from_cache now unpacks widen_stereo tuple correctly.
Process

## MAIN

In [2]:
# MAIN: end-to-end sanity test for I/O → Analysis → DSP Primitives → Processors
# Run this cell after loading the previous cells.
# It will:
#  - create a workspace
#  - import your mix (set MIX_SRC below)
#  - analyze the original
#  - build a preview cache
#  - render a few dial presets
#  - save outputs, register artifacts
#  - compare metrics in a table
#  - plot spectrum & loudness overlays
#
# Safe to re-run; it creates a new timestamped workspace each time.

import os, numpy as np, matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd


# ---------- 1) SET YOUR SOURCE FILE HERE ----------
MIX_SRC = "/Users/itay/Documents/muxing/ITAY - CRASHING v.6 MIX ONLY.wav"   # <-- change me
MIX_SRC = "/Users/itay/Documents/muxing/ITAY - 4 CHORDS v.21 STEFAN.wav"   # <-- change me
MIX_SRC = "/Users/itay/Documents/post_mix_data/input_mix_raw/ITAY - HOTEL POLYPHONIA STEFAN.wav"   # <-- change me



In [6]:
# ============================================
# MAIN — Premasters Only (Multi-Song / Multi-Mix Mode)
# ============================================

import os, soundfile as sf
from dataclasses import asdict

# ---------- 0) SET YOUR SONG ROOT FOLDER ----------
MIXES_ROOT = "/Users/itay/Documents/post_mix_data/mixes"
OUTPUT_ROOT = "/Users/itay/Documents/post_mix_data/PostMixRuns"

# ---------- 1) Global toggles ----------
DO_STREAMING_SIM = False
RECS_LIMIT       = 3
REPORT_REF_NAME  = "Original"

# ---------- 2) Walk through song folders ----------
for song_name in os.listdir(MIXES_ROOT):
    song_path = os.path.join(MIXES_ROOT, song_name)
    if not os.path.isdir(song_path):
        continue  # skip non-folders

    print(f"\n=== Processing song: {song_name} ===")

    # go over all mixes for this song
    for fname in os.listdir(song_path):
        if not fname.lower().endswith(".wav"):
            continue
        MIX_SRC = os.path.join(song_path, fname)

        # project name = version filename without extension
        PROJECT = os.path.splitext(fname)[0]
        print(f"\n--- Mix version: {PROJECT} ---")

        # ---------- Workspace, manifest, logger ----------
        song_out_root = os.path.join(OUTPUT_ROOT, song_name)
        os.makedirs(song_out_root, exist_ok=True)

        paths = make_workspace(base_dir=song_out_root, project=PROJECT)
        man   = Manifest(project=PROJECT, workspace=paths)
        logger = RunLogger.start(song_out_root, tag=PROJECT)

        env = capture_environment()
        logger.log_event("env", {"environment": env})

        # ---------- Import ----------
        if not os.path.exists(MIX_SRC):
            print(f"❌ File not found: {MIX_SRC}")
            continue
        mix_path = import_mix(paths, MIX_SRC, alias="mix.wav")
        register_input(man, mix_path, alias="mix")

        # ---------- Analyze ----------
        x, sr = sf.read(mix_path)
        ensure_audio_valid(x, "mix")

        buf = load_wav(mix_path)
        print_audio_summary(buf, "Original Mix")

        rep_orig = analyze_wav(mix_path)

        # ---------- Recommendations ----------
        recs = recommend_from_analysis(rep_orig)
        print(recommendation_summary(recs))

        recommended_plan = build_premaster_plan_from_recs(recs, limit=RECS_LIMIT, prefix="PM")

        variants_plan = []
        for v in ALL_VARIANTS:
            dials = DialState(**v["dial_state"]) if isinstance(v, dict) else v[1]
            name  = v["name"] if isinstance(v, dict) else v[0]
            if any(name == r[0] for r in recommended_plan):  # tag only real recommendations
                name = f"{name}_recommended"
            variants_plan.append((name, dials))

        # ---------- Render Premasters ----------
        engine = RenderEngine(x, sr, preprocess=PreprocessConfig(
            low_cutoff=120.0, kick_lo=40.0, kick_hi=110.0
        ))
        pre_meta = engine.preprocess()

        opts = RenderOptions(target_peak_dbfs=CFG.render_peak_target_dbfs,
                             bit_depth=CFG.default_bit_depth,
                             hpf_hz=None, save_headroom_first=False)

        var_metas = engine.render_variants(paths.premastered, variants_plan, opts=opts)

        for meta in var_metas:
            register_and_log_artifact(man, logger, meta["out_path"], kind="premastered",
                                      params=meta, stage=f"variant__{os.path.basename(meta['out_path'])}")

        # ---------- STOP HERE ----------
        print("\n=== DONE ===")
        print("Workspace:", paths.root)
        print("Premastered variants:", *[m["out_path"] for m in var_metas], sep="\n - ")



=== Processing song: beyond ===

--- Mix version: ITAY - BEYOND v.17 ---
Workspace created at: /Users/itay/Documents/post_mix_data/PostMixRuns/beyond/ITAY - BEYOND v.17_20250821-062646
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/beyond/ITAY - BEYOND v.17_20250821-062646/inputs/mix.wav
Original Mix: sr=44100 | ch=2 | dur=242.0s | peak=0.622549 | rms=0.072326
  path: /Users/itay/Documents/post_mix_data/PostMixRuns/beyond/ITAY - BEYOND v.17_20250821-062646/inputs/mix.wav
  sha256: b00ece6229795f94...
  src dtype: int32 | src ch: 2
- Punch Up the Transients (priority 2): B16 P38 C10 A14 W6
    • crest 18.0 dB (dynamic) and LUFS -24.6 (quiet) → add transient definition
    • small air lift to help intelligibility

=== DONE ===
Workspace: /Users/itay/Documents/post_mix_data/PostMixRuns/beyond/ITAY - BEYOND v.17_20250821-062646
Premastered variants:
 - /Users/itay/Documents/post_mix_data/PostMixRuns/beyond/ITAY - BEYOND v.17_20250821-062646/premastered/Transparent_recommen